<a href="https://colab.research.google.com/github/amitgundad/pySpark/blob/master/2_Admission_Prediction_with_PySpark_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Dependencies & Run a SparkSession


In [ ]:
#install pyspark
! pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 50.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=05aec9039e81931bf99632b04dbcbe4c2e5487a9ead8777899ae86b83c367faf
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
#create a sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('spark').getOrCreate()

Clone & Explore dataset

In [ ]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [ ]:
#check the presence of dataset
! ls admission_dataset

Admission_Predict_Ver1.1.csv


In [ ]:
#create a spark dataframe
df = spark.read.csv('/content/admission_dataset/Admission_Predict_Ver1.1.csv',header=True,inferSchema=True)
df.head(5)

[Row(Serial No=1, GRE Score=337, TOEFL Score=118, University Rating=4, SOP=4.5, LOR=4.5, CGPA=9.65, Research=1, Chance of Admit=0.92),
 Row(Serial No=2, GRE Score=324, TOEFL Score=107, University Rating=4, SOP=4.0, LOR=4.5, CGPA=8.87, Research=1, Chance of Admit=0.76),
 Row(Serial No=3, GRE Score=316, TOEFL Score=104, University Rating=3, SOP=3.0, LOR=3.5, CGPA=8.0, Research=1, Chance of Admit=0.72),
 Row(Serial No=4, GRE Score=322, TOEFL Score=110, University Rating=3, SOP=3.5, LOR=2.5, CGPA=8.67, Research=1, Chance of Admit=0.8),
 Row(Serial No=5, GRE Score=314, TOEFL Score=103, University Rating=2, SOP=2.0, LOR=3.0, CGPA=8.21, Research=0, Chance of Admit=0.65)]

In [ ]:
#display dataframe
df.show(5)

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
only showing top 5 rows



In [ ]:
#get the no.of rows & columns
print((df.count(),len(df.columns)))

(500, 9)


In [ ]:
#print schema
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [ ]:
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

Data Cleaning

In [ ]:
#drop the unnecessary column
df = df.drop('Serial No')

In [ ]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [ ]:
#check for null values
for i in df.columns:
  print(i + ":", df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


Correlation Analysis & Feature Selection

In [ ]:
# correlation analysis
for col in df.columns:
  print('Correlation to chance of admit col for {} is {}'.format(col, df.stat.corr('Chance of Admit', col)))

Correlation to chance of admit col for GRE Score is 0.8103506354632598
Correlation to chance of admit col for TOEFL Score is 0.7922276143050823
Correlation to chance of admit col for University Rating is 0.6901323687886892
Correlation to chance of admit col for SOP is 0.6841365241316723
Correlation to chance of admit col for LOR is 0.6453645135280112
Correlation to chance of admit col for CGPA is 0.882412574904574
Correlation to chance of admit col for Research is 0.5458710294711379
Correlation to chance of admit col for Chance of Admit is 1.0


In [ ]:
# feature selection
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['GRE Score','TOEFL Score','CGPA'], outputCol = 'Features')

In [ ]:
#display dataframe
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          Features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

Build the Linear Regression Model

In [ ]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select('Features','Chance of Admit')

In [ ]:
#print schema of final data
final_data.printSchema()

root
 |-- Features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [ ]:
#split the dataset into training and testing set
train, test = final_data.randomSplit([0.7,0.3])

In [ ]:
#build & train the model
models = LinearRegression(featuresCol='Features',labelCol='Chance of Admit')
model = models.fit(train)

In [ ]:
#get summary of the model
summary = model.summary

In [ ]:
#print the rmse & r2 score
print("RMSE : ",summary.rootMeanSquaredError)
print("R2 Score : ",summary.r2)

RMSE :  0.0608939856062807
R2 Score :  0.8171261176288201


Evaluate & Save the Model

In [ ]:
#transform on the test data
predictions = model.transform(test)

In [ ]:
#display the predictions
predictions.show()

+------------------+---------------+-------------------+
|          Features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
|  [293.0,97.0,7.8]|           0.64| 0.5215072638440936|
| [295.0,96.0,7.34]|           0.47|0.45866821050976614|
|  [296.0,97.0,7.8]|           0.49| 0.5292091632181923|
| [297.0,96.0,7.89]|           0.43| 0.5413780710443923|
| [298.0,99.0,7.46]|           0.53|  0.492573198090706|
|  [298.0,99.0,7.6]|           0.46| 0.5123196281511879|
|[298.0,100.0,7.95]|           0.58| 0.5647782288778693|
| [299.0,96.0,7.86]|           0.54| 0.5422812927570211|
|[299.0,100.0,8.02]|           0.63| 0.5772187436994762|
|[299.0,102.0,8.62]|           0.56| 0.6680313522524941|
| [300.0,95.0,8.22]|           0.62|  0.592532601414151|
| [301.0,97.0,7.88]|           0.44| 0.5533293364952983|
| [301.0,99.0,8.22]|           0.64|  0.607470003507421|
|[301.0,104.0,8.12]|           0.68| 0.6088280384844571|
| [302.0,99.0,7.45]|           

In [ ]:
#evaluate the model 
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(predictionCol='prediction', labelCol = 'Chance of Admit', metricName = 'r2')
print("R2 on the test data ",eval.evaluate(predictions))

R2 on the test data  0.7716175290967273


In [ ]:
#save the model
#model.save("OurModel")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load the model
#from pyspark.ml.regression import LinearRegressionModel
#model = LinearRegressionModel.load("OurModel")